<a href="https://colab.research.google.com/github/gugi200/final_project/blob/main/comp_diff_models_custom_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing libraries

In [1]:
!pip install wandb -qU
!wandb login 3014974e724f01c4d63f956fa13fd7f0463e16d4
!pip install torchmetrics
!pip install mlxtend>=0.19.0

!pip list | grep mlx


#
#   Michael Gugala
#   02/12/2023
#   Image recognition
#   Master 4th year project
#   Univeristy of Bristol
#

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn

import torchvision
from torchvision import datasets#
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchmetrics
from torchvision.models import resnet50, ResNet50_Weights
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.utils import Bunch

from PIL import Image

import requests
import random
import shutil
import zipfile
from pathlib import Path
from io import BytesIO, StringIO
import os

import wandb
import cv2
from timeit import default_timer as timer
from tqdm.auto import tqdm

# check imports
print(torch.__version__)
print(torchvision.__version__)

#agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 11.3 MB/s eta 0:00:00
mlxtend                          0.22.0
2.1.0+cu121
0.16.0+cu121


'cuda'

# downloading data and spittiling into raw train and test subsets

In [2]:
! rm -R custom_dataset train_raw test_raw

rm: cannot remove 'custom_dataset': No such file or directory
rm: cannot remove 'train_raw': No such file or directory
rm: cannot remove 'test_raw': No such file or directory


In [3]:
colabPath = Path("custom_dataset")

#  Create a dir
if colabPath.is_dir():
    print('directory already exists')
else:
    colabPath.mkdir(parents=True, exist_ok=True)

# download zipped data
    github_path = "https://github.com/gugi200/final_project/raw/main/data.zip"
    # file = "dataset_pressure_sensor.zip"
    file = 'data.zip'
with open(colabPath / file, "wb") as f:
    request = requests.get(github_path)
    f.write(request.content)


# unzip the data
with zipfile.ZipFile(colabPath / file, "r") as f:
    f.extractall(colabPath)




TRAIN_RATIO = 0.75
rawDataPath = colabPath/"data_sensor"
dirs = os.listdir(colabPath/"data_sensor")


#  Create a dir for train and test data
extendedTrain = Path("train_raw")
extendedTest = Path("test_raw")
if extendedTrain.is_dir():
    print('directory already exists')
else:
    extendedTrain.mkdir(parents=True, exist_ok=True)
    extendedTest.mkdir(parents=True, exist_ok=True)
    for dir in dirs:
        path = extendedTrain / dir
        path.mkdir(parents=True, exist_ok=True)
    for dir in dirs:
        path = extendedTest / dir
        path.mkdir(parents=True, exist_ok=True)

for dir in dirs:
    files = os.listdir(rawDataPath / dir)
    length = int(TRAIN_RATIO*len(files))
    random.shuffle(files)

    train_set = files[:length]
    test_set = files[length:]

    for data in train_set:
        shutil.copy(rawDataPath / dir / data, extendedTrain / dir / data)

    for data in test_set:
        shutil.copy(rawDataPath / dir / data, extendedTest / dir / data)

l = 0
for dir in dirs:
    l += len(os.listdir(extendedTrain/dir))
    print(dir, len(os.listdir(extendedTrain/dir)))
print(l)

l = 0
for dir in dirs:
    l += len(os.listdir(extendedTest/dir))
    print(dir, len(os.listdir(extendedTest/dir)))
print(l)




import os
from PIL import Image
from PIL import UnidentifiedImageError

for folder_path in [extendedTrain, extendedTest, Path(colabPath/'data_sensor')]:
    extensions = []
    for fldr in os.listdir(folder_path):
        sub_folder_path = os.path.join(folder_path, fldr)
        for filee in os.listdir(sub_folder_path):
            file_path = os.path.join(sub_folder_path, filee)
            try:
                im = Image.open(file_path)
            except UnidentifiedImageError:
                print(file_path)
                os.remove(file_path)

big_fizzy 775
h_bottle 786
mug 821
small_fizzy 722
hand 750
h_big_bottle 839
nothing 229
can 748
5670
big_fizzy 259
h_bottle 263
mug 274
small_fizzy 241
hand 251
h_big_bottle 280
nothing 77
can 250
1895
train_raw/big_fizzy/sensor_6431.jpg
train_raw/big_fizzy/sensor_6425.jpg
train_raw/big_fizzy/sensor_6427.jpg
train_raw/big_fizzy/sensor_6432.jpg
train_raw/big_fizzy/sensor_6430.jpg
train_raw/big_fizzy/sensor_6428.jpg
train_raw/big_fizzy/sensor_6434.jpg
train_raw/big_fizzy/sensor_6435.jpg
train_raw/can/sensor_2781.jpg
train_raw/can/sensor_2792.jpg
train_raw/can/sensor_2798.jpg
train_raw/can/sensor_2793.jpg
train_raw/can/sensor_2787.jpg
train_raw/can/sensor_2790.jpg
train_raw/can/sensor_2797.jpg
train_raw/can/sensor_2799.jpg
train_raw/can/sensor_2785.jpg
train_raw/can/sensor_2794.jpg
train_raw/can/sensor_2789.jpg
train_raw/can/sensor_2782.jpg
train_raw/can/sensor_2786.jpg
train_raw/can/sensor_2796.jpg
train_raw/can/sensor_2783.jpg
test_raw/big_fizzy/sensor_6426.jpg
test_raw/big_fizzy/senso

# Library

In [4]:


# create data set from a custom data
def create_dataset(path, batchsize, preprocess=None, mean=None, std=None):
    '''
    input:
    path - path to the folder with the data
           eg for train - "data/FashionMNIST/train"
    batchsize - eg 32
    mean (optional)- for normalization eg. [0.25, 0.25, 0.25]
    std (optional)- for nortmalization eg [0.1, 0.1, 0.1]

    returns:
    dataloader with image size of 224
    class_names
    '''
    if not preprocess:

        if mean:
            preprocess = transforms.Compose([

                transforms.Resize(size=(224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ])
        else:
            preprocess = transforms.Compose([

                transforms.Resize(size=(224, 224)),
                transforms.ToTensor()
            ])


    data = datasets.ImageFolder(root=Path(path),
                                    transform=preprocess, # tranform for the data
                                    target_transform=None) # transform for label
    dataloader = DataLoader(dataset=data,
                                batch_size=batchsize,
                                shuffle=True) #  shuffling to remove order
    class_names = data.classes
    return dataloader, class_names, data.targets

# visdualize 9 random images in a batch
def visualise_data(dataloader, class_names, batchsize):
    '''
    input dataloader
    class names

    displays 9 random images in a batch and their labels
    '''
    train_features_batch, train_labels_batch = next(iter(dataloader))
    print("length of data: ", len(train_features_batch), 'length of labels: ', len(train_labels_batch))
    # display random datapoints
    fig = plt.figure(figsize=(9, 9))
    rows, cols  = 3, 3
    for pic in range(1, 1+rows*cols):
        rand_int = np.random.randint(0, batchsize)
        img = train_features_batch[rand_int]
        img_RGB = img.permute([1, 2, 0]).numpy()
        fig.add_subplot(rows, cols, pic)
        plt.imshow(img_RGB.squeeze())
        plt.axis(False)
        plt.title(class_names[train_labels_batch[rand_int]])


# create optimizer
def create_optiimizer(model, optimizer, lr):
    '''
    inputs:
    model - CNN network
    optimizer - "adam" or "sgd"
    lr - learning rate eg. 0.01

    '''
    if optimizer=='adam':
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=lr
                                     )
    elif optimizer=='sgd':
        optimizer = torch.optim.SGD(model.parameters(),
                                     lr=lr,
                                    momentum=0.9
                                     )

    return optimizer


def get_lossFn():
    '''
        returns CrossEntropyLoss function
    '''

    return nn.CrossEntropyLoss()

# create train step
def train_step(model, metric, loss_fn, optimizer,
               data_loader, device, debug=False, wnb=True):
    '''
    model - CNN network
    metric - metric to calculate accuracy
    loss_fn - loss function
    optimizer - optimizer to be applied
    data_loader - dataloader
    device - decide for the model to train
    debug (optional)- if True prints average loss and metric of the batch

    returns
    train_loss - average loss of the batch
    train_acc - average metric score of the batch

    The function saves the metric score and loss of each iteration in WandB

    '''
    train_loss, train_acc = 0, 0

    model.train()
    for batch, (X, y) in enumerate(data_loader):
        # put data on the device
        X, y = X.to(device), y.to(device)

        #forward pass, return raw logits
        y_pred = model(X)

        #loss
        loss = loss_fn(y_pred, y)
        #accuracy
        acc = metric(y, torch.argmax(y_pred, dim=1))

        train_loss += loss # accumulate train loss
        train_acc += acc # accumulate train accuracy

        # zero grad
        optimizer.zero_grad()

        # loss bacward
        loss.backward()

        #optimizer step
        optimizer.step()

        # log in wandb
        if wnb:
            wandb.log({"loss": loss,
                       'accuracy': acc})

    # device total loss and accuracy by length of train dataloader
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    if debug:
        print(f'Train loss: {train_loss:.4f}, Train acc: {train_acc*100:0.4f}%')

    return train_loss, train_acc


# create test step
def test_step(model, metric, loss_fn, data_loader, device, debug=False, wnb=True):
    '''
    model - CNN network
    metric - metric to calculate accuracy
    loss_fn - loss function
    data_loader - dataloader
    device - decide for the model to train
    debug (optional)- if True prints average loss and metric of the batch

    returns
    test_loss - average loss of the batch
    test_acc - average metric score of the batch

    The function saves the metric score and loss of each iteration in WandB

    '''
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X_test, y_test in data_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            #1 forward pass
            test_pred = model(X_test)

            # calculate loss
            loss = loss_fn(test_pred, y_test)
            test_loss += loss

            #accuracy
            acc = metric(y_test, test_pred.argmax(dim=1))
            test_acc += acc

            if wnb:
                wandb.log({"test loss": loss,
                           'test accuracy': acc})

        # Calculate the test loss average batch
        test_loss /= len(data_loader)

        # acc per bactch
        test_acc /= len(data_loader)

        # Print out what's happening
        if debug:
            print(f'Test loss: {test_loss:.4f}  |  Test acc: {test_acc*100:.4f}%')

        return test_loss, test_acc

# create evaluation loop
def eval_model(model: torch.nn.Module,
                data_loader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module,
                accuracy_fn,
               device):
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in tqdm(data_loader):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)

            #accumulate the loss and acc
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y, y_pred.argmax(dim=1))

        # ave loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works if a model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc.item()*100}




def visualize_preds(model, dataloader, class_names, batchsize):
    plt.figure(figsize=(9, 9))
    nrows = 3
    ncols = 3
    model = model.cpu()

    dataL_len = len(dataloader)
    data = iter(dataloader)
    for i in range(3):
        model.eval()
        with torch.inference_mode():

            X, y = next(data)
            X, y = X.cpu(), y.cpu()
            for j in range(3):
                randint = np.random.randint(0, batchsize)
                X_sample, y_sample = X[randint], y[randint]
                pred_logit = model(X_sample.unsqueeze(dim=0))

                pred_prob = pred_logit.argmax(dim=1)


                plt.subplot(nrows, ncols, (3*i)+j+1);
                plt.imshow(X_sample.squeeze().permute([1, 2, 0]), cmap='gray');

                #find pred_label in text form
                pred_label = class_names[pred_prob];

                # find truth label
                truth_label = class_names[y_sample];

                title_text = f'Pred: {pred_label}  \n  Truth: {truth_label}'

                if pred_label==truth_label:
                    plt.title(title_text, fontsize=10, c='g');
                else:
                    plt.title(title_text, fontsize=10, c='r');
                plt.axis(False)
                plt.tight_layout()



def plot_decision_matrix(class_names, y_pred_tensor, targets):
    # setup confusion matrix
    confmat = ConfusionMatrix(num_classes=len(class_names), task='multiclass')
    confmat_tensor = confmat(preds=y_pred_tensor,
                            target=targets)

    # plot consufionmatrix
    fig, ax = plot_confusion_matrix(
        conf_mat=confmat_tensor.numpy(),
        class_names=class_names,
        figsize=(10, 7)
    )



def make_predictions_dataloader(model, dataloader, device, class_names):
    preds = []
    target = []
    model = model.to(device)
    model.eval()
    test_acc = 0
    metric = torchmetrics.classification.Accuracy(
        task="multiclass",
        num_classes=len(class_names)
    ).to(device)
    with torch.inference_mode():

        for X_test, y_test in tqdm(dataloader):
            X_test, y_test = X_test.to(device), y_test.to(device)
            batch_pred = model(X_test)
            batch_pred = batch_pred.cpu()
            y_test = y_test.cpu()
            preds.append(np.array(batch_pred.argmax(dim=1)))
            target.append(y_test)

            acc = metric(y_test, batch_pred.argmax(dim=1))
            test_acc += acc

        # acc per bactch
        test_acc /= len(dataloader)


    return np.concatenate(preds), np.concatenate(target), test_acc


def make_predictions(model, data, device):
    model.eval()
    data = data.to(device)
    model = model.to(device)
    with torch.inference_mode():
        y_preds = model(data)
    return y_preds.cpu()


def dataloader_to_numpy(dataloader):
    for i, (data, target) in enumerate(dataloader):
        if i==0:
            data_numpy = data.numpy()
            target_numpy = target.numpy()
        else:
            data_numpy = np.append(data_numpy, data.numpy(), axis=0)
            target_numpy = np.append(target_numpy, target.numpy(), axis=0)
    return data_numpy, target_numpy


def get_datalodaer(batchsize, train_path, test_path):

    train_dataloader, class_names, _ = create_dataset(
                                        path=train_path,
                                        batchsize=batchsize,
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]
                                    )
    test_dataloader, _, _ = create_dataset(
                                path=test_path,
                                batchsize=batchsize,
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]
                                    )
    return train_dataloader, test_dataloader, class_names


def train_test_loop(config, model, train_dataloader, test_dataloader,
                    class_names, device):

    prev_acc = 0
    loss_fn = get_lossFn()
    optimizer = create_optiimizer(model=model,
                                    optimizer=config.optimizer,
                                    lr=config.learning_rate
    )
    metric = torchmetrics.classification.Accuracy(
        task="multiclass",
        num_classes=len(class_names)
    ).to(device)
    train_time_start = timer()
    for epoch in range(config.epochs):

        print([g['lr'] for g in optimizer.param_groups])

        ave_batch_train_loss, ave_batch_train_acc = train_step(
            model=model,
            metric=metric,
            loss_fn=loss_fn,
            optimizer=optimizer,
            data_loader=train_dataloader,
            device=device,
            debug=True
        )
        ave_batch_test_loss, ave_batch_test_acc = test_step(
            model=model,
            metric=metric,
            loss_fn=loss_fn,
            data_loader=test_dataloader,
            device=device,
            debug=True
        )
        curr_acc = ave_batch_train_acc

        if ((curr_acc-prev_acc)<0.03) and (curr_acc>0.5):
            for g in optimizer.param_groups:
              g['lr'] = 0.001
        prev_acc = curr_acc

        wandb.log({"average train batch loss": ave_batch_train_loss,
                    "average train batch metric": ave_batch_train_acc,
                    "average test batch loss": ave_batch_test_loss,
                    "average test batch metric": ave_batch_test_acc,
                    "epoch": epoch
                    })
    train_time_end = timer()
    wandb.log({"train time": train_time_end - train_time_start})
    return


def train_test_loop_nonpipe(model, train_dataloader, test_dataloader, optim, lr,
                    epochs, class_names, device):
    loss_fn = get_lossFn()
    optimizer = create_optiimizer(model=model,
                                    optimizer=optim,
                                    lr=lr
    )
    metric = torchmetrics.classification.Accuracy(
        task="multiclass",
        num_classes=len(class_names)
    ).to(device)
    train_time_start = timer()
    for epoch in range(epochs):
        ave_batch_loss, ave_batch_metric = train_step(
            model=model,
            metric=metric,
            loss_fn=loss_fn,
            optimizer=optimizer,
            data_loader=train_dataloader,
            device=device,
            debug=True
        )
        ave_batch_loss, ave_batch_metric = test_step(
            model=model,
            metric=metric,
            loss_fn=loss_fn,
            data_loader=test_dataloader,
            device=device,
            debug=True
        )
        wandb.log({"average train batch loss": ave_batch_loss,
                    "average train batch metric": ave_batch_metric,
                    "average test batch loss": ave_batch_loss,
                    "average test batch metric": ave_batch_metric,
                    "epoch": epoch
                    })
    train_time_end = timer()
    wandb.log({"train time": train_time_end - train_time_start})


# models

## setup wandb

In [5]:

torch.manual_seed(42)
torch.cuda.manual_seed(42)



def softThreshold(lower, upper):
    def func(data):
        data = np.asarray(data)
        th = np.where(data>upper, 255, data)
        th = np.where(th<lower, 0, th).astype(np.uint8)
        return Image.fromarray(th)
    return func

def expMapping(alpha):
    def func(data):
            data = np.asarray(data)
            data = np.where(data==0, 1, data)
            mapped = np.exp( -( alpha/(data) ) )*255
            mapped = mapped.astype(np.uint8)
            return Image.fromarray(mapped)
    return func



sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'
    }
sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [8]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 5}
    })

# parameters_dict.update({
#     'learning_rate': {
#         # a flat distribution between 0 and 0.1
#         'distribution': 'uniform',
#         'min': 0,
#         'max': 0.1
#       },
#     'batch_size': {
#         # integers between 32 and 256
#         # with evenly-distributed logarithms
#         'distribution': 'q_log_uniform_values',
#         'q': 8,
#         'min': 8,
#         'max': 32,
#       }
#     })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'values': [0.01]
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        'values': [16, 32]
      }
    })



import pprint
pprint.pprint(sweep_config)




{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [16, 32]},
                'epochs': {'value': 5},
                'fc_layer_size': {'values': [8]},
                'learning_rate': {'values': [0.01]},
                'optimizer': {'values': ['adam', 'sgd']}}}


## restnet50

In [ ]:

def train_model_resnet50(config=None):
    with wandb.init(config=config):

        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)

        model = resnet50(
            weights=ResNet50_Weights.DEFAULT).to(device)
        model.fc = nn.Linear(2048 , config.fc_layer_size).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# restnet pipeline
sweep_id_restnet50 = wandb.sweep(sweep_config, project="restnet-customData")
wandb.agent(sweep_id_restnet50, train_model_resnet50)

Create sweep with ID: kxplw9tn
Sweep URL: https://wandb.ai/gugi200/restnet-customData/sweeps/kxplw9tn


wandb: Agent Starting Run: j91fxy4n with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


[0.01]
Train loss: 1.1293, Train acc: 56.8107%
Test loss: 0.6669  |  Test acc: 75.7037%
[0.01]
Train loss: 0.6342, Train acc: 77.5472%
Test loss: 0.5126  |  Test acc: 81.5527%
[0.01]
Train loss: 0.4512, Train acc: 84.4511%
Test loss: 0.5980  |  Test acc: 81.9764%
[0.01]
Train loss: 0.3494, Train acc: 87.9910%
Test loss: 0.4218  |  Test acc: 86.9022%
[0.01]
Train loss: 0.2483, Train acc: 91.5191%
Test loss: 0.2214  |  Test acc: 92.6831%


accuracy,▂▂▁▅▆▄▅▅▄█▇▅▆▆▇▅▇▅▅█▅▇▆▇▇▅█▇▇▇▇█▇▇▇█▇▇▇█
average test batch loss,█▆▇▄▁
average test batch metric,▁▃▄▆█
average train batch loss,█▄▃▂▁
average train batch metric,▁▅▇▇█
epoch,▁▃▅▆█
loss,█▇█▄▃▆▅▆▄▂▃▄▃▂▂▆▂▄▄▁▄▂▃▃▂▅▁▅▂▁▂▁▂▂▂▁▄▂▂▁
test accuracy,▁▅▂▃▅▆▅▇▅▅▂▂▇▆▆▆▂▇█▆▂▃▆▃█▆▅▅▆▆▇▆▆▇████▇▆
test loss,█▃▃▄▃▃▃▂▃▅▄▅▂▃▃▂▄▂▁▂▄▄▂▃▁▂▃▄▂▂▂▃▂▁▁▁▁▁▂▂
train time,▁
accuracy,1.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: z7qpogz2 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


...

accuracy,▂▁▅▄▅▄▄▆▆▆▄▅▆▆▆▅▅▆▆▇▆▆▇▇▇▆█▇▇██▇████▆██▇
average test batch loss,▇█▃▁▁
average test batch metric,▂▁▆█▇
average train batch loss,█▄▃▁▁
average train batch metric,▁▅▆██
epoch,▁▃▅▆█
loss,█▇▅▅▅▅▅▅▄▃▄▄▄▃▃▇▅▃▃▂▃▄▂▃▂▂▁▂▂▁▁▁▂▂▁▁▃▁▂▂
test accuracy,▅▄▁▃▃▄▂▂▂▃▂▃▃▂▂▁▃▇▆▅▅▅▃▆▇▅▆▇▇███▆▅▆▆▆█▇▇
test loss,▄▅▅▅▄▄▅▅█▅▄▅▆▄▆▆▄▂▂▂▃▄▃▂▁▂▂▂▁▂▁▁▂▃▃▂▂▁▂▂
train time,▁
accuracy,0.86667


wandb: Agent Starting Run: 5rcf8j90 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


[0.01]
Train loss: 0.7267, Train acc: 74.2561%
Test loss: 0.2804  |  Test acc: 91.0840%
[0.01]
Train loss: 0.1318, Train acc: 95.6921%
Test loss: 0.1374  |  Test acc: 95.9146%
[0.01]
Train loss: 0.0540, Train acc: 98.4110%
Test loss: 0.0840  |  Test acc: 97.7684%
[0.001]
Train loss: 0.0263, Train acc: 99.3621%
Test loss: 0.0613  |  Test acc: 98.4075%
[0.001]
Train loss: 0.0137, Train acc: 99.7858%
Test loss: 0.0623  |  Test acc: 98.4110%


accuracy,▁▂▄▆▆▆▆▆▇█▇██▇██████████████████████████
average test batch loss,█▃▂▁▁
average test batch metric,▁▆▇██
average train batch loss,█▂▁▁▁
average train batch metric,▁▇███
epoch,▁▃▅▆█
loss,█▇▅▃▃▂▃▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test accuracy,▇▅▅▇█▂▁▅▆▇▆█▆▅▇▇███▇██▇▇████▇▇█▇█▇█▆█▇▆█
test loss,▂▄▃▃▁▄█▄▂▁▂▁▂▄▂▂▁▁▁▁▂▂▁▂▁▁▁▁▁▁▁▃▁▁▁▂▁▂▂▁
train time,▁
accuracy,0.93333


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## alexnet

In [ ]:
def train_model_alexnet(config=None):
    with wandb.init(config=config):

        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)

        weight = list(torchvision.models.get_model_weights('alexnet'))[-1]
        model = torch.hub.load('pytorch/vision', 'alexnet', weight).to(device)
        model.classifier[6] = nn.Linear(4096 , config.fc_layer_size).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# alexnet pipeline
sweep_id_alexnet = wandb.sweep(sweep_config, project="alexnet-customData")
wandb.agent(sweep_id_alexnet, train_model_alexnet)

Create sweep with ID: rs2ja1ip
Sweep URL: https://wandb.ai/gugi200/alexnet-customData/sweeps/rs2ja1ip


wandb: Agent Starting Run: m2zmxyet with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 85.7MB/s]


[0.01]
Train loss: 204.4612, Train acc: 13.9542%
Test loss: 2.0941  |  Test acc: 12.7724%
[0.01]
Train loss: 2.0638, Train acc: 13.6874%
Test loss: 2.0406  |  Test acc: 14.8532%
[0.001]
Train loss: 2.0395, Train acc: 14.5597%
Test loss: 2.0365  |  Test acc: 14.8381%
[0.001]
Train loss: 2.0374, Train acc: 14.6128%
Test loss: 2.0369  |  Test acc: 14.8381%
[0.001]
Train loss: 2.0379, Train acc: 14.4688%
Test loss: 2.0386  |  Test acc: 14.8456%


accuracy,▂▅▂▇▂▃▃▃▂▅▃▃▃▂▅▃▅▃▂▅▂▃▃▁▃▇▃▃▃▅▅▁▆▂▂█▂▅▃▅
average test batch loss,█▂▁▁▁
average test batch metric,▁████
average train batch loss,█▁▁▁▁
average train batch metric,▃▁██▇
epoch,▁▃▅▆█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test accuracy,▂▄▄▂▂▇▁▂▅▂▂▄▄▁▄▄▅▂▅▅▇▂▅▂█▂▁▂▂▅▂▁▅▂▅█▅▄▄▂
test loss,▂▄█▆▆▃▆▇▃▅▄▄▄▄▂▂▃▅▂▃▂▅▂▂▁▅▅▂▂▁▅▃▂▂▄▃▁▃▁▃
train time,▁
accuracy,0.2


...

wandb: Agent Starting Run: 1ei21v50 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]
Train loss: nan, Train acc: 13.8465%
Test loss: nan  |  Test acc: 13.4640%
[0.01]
Train loss: nan, Train acc: 13.6217%
Test loss: nan  |  Test acc: 13.4534%
[0.001]
Train loss: nan, Train acc: 13.5617%
Test loss: nan  |  Test acc: 13.4675%
[0.001]
Train loss: nan, Train acc: 13.6017%
Test loss: nan  |  Test acc: 13.4710%
[0.001]
Train loss: nan, Train acc: 13.5617%
Test loss: nan  |  Test acc: 13.4605%


accuracy,▅▁▃▁▅▃▅▁▂█▄▃▄▁▃▅▃▂▁▅▁▆▂▄▅▆▄▃▂▁▅▂▃▃▃▂▃▂▅▄
average test batch metric,▅▁▇█▄
average train batch metric,█▂▁▂▁
epoch,▁▃▅▆█
loss,▁
test accuracy,▅▃▃▄▅▆▃▃▆▅▃▁▄▆▄▃▅▄▆▃▃▄▄▇▅▃▄▅▄█▄▄▃▆▆▃▃▇▃▆
train time,▁
accuracy,0.06667
average test batch loss,nan
average test batch metric,0.1346
average train batch loss,nan


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## convnext base

In [ ]:
def train_model_convnext_base(config=None):
    with wandb.init(config=config):
        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)

        weight = list(torchvision.models.get_model_weights('convnext_base'))[-1]
        model = torch.hub.load('pytorch/vision', 'convnext_base', weight).to(device)
        model.classifier[2] = nn.Linear(1024 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model

# convnext_base pipeline
sweep_id_convnext_base = wandb.sweep(sweep_config, project="convnext_base-customData")
wandb.agent(sweep_id_convnext_base, train_model_convnext_base)

## densenet161

In [ ]:
def train_model_densenet161(config=None):
    with wandb.init(config=config):
        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('densenet161'))[-1]
        model = torch.hub.load('pytorch/vision', 'densenet161', weight).to(device)
        model.classifier = nn.Linear(2208 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model

# densenet161 pipeline
sweep_id_densenet161 = wandb.sweep(sweep_config, project="densenet161-customData")
wandb.agent(sweep_id_densenet161, train_model_densenet161)

## efficientnet_v2_1

In [ ]:
def train_model_efficientnet_v2_l(config=None):
    with wandb.init(config=config):
        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('efficientnet_v2_l'))[-1]
        model = torch.hub.load('pytorch/vision', 'efficientnet_v2_l', weight).to(device)
        model.classifier[1] = nn.Linear(1280 , len(class_names), bias=True).to(device).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# efficientnet_v2_l pipeline
sweep_id_efficientnet_v2_l = wandb.sweep(sweep_config, project="efficientnet_v2_l-customData")
wandb.agent(sweep_id_efficientnet_v2_l, train_model_efficientnet_v2_l)

## googlenet

In [ ]:
def train_model_googlenet(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('googlenet'))[-1]
        model = torch.hub.load('pytorch/vision', 'googlenet', weight).to(device)
        model.fc = nn.Linear(1024 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# googlenet pipeline
sweep_id_googlenet = wandb.sweep(sweep_config, project="googlenet-customData")
wandb.agent(sweep_id_googlenet, train_model_googlenet)

Create sweep with ID: m3uz7d5c
Sweep URL: https://wandb.ai/gugi200/googlenet-customData/sweeps/m3uz7d5c


wandb: Agent Starting Run: 54ttrmsb with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:03<00:00, 16.1MB/s]


[0.01]
Train loss: 1.1049, Train acc: 59.8796%
Test loss: 0.8180  |  Test acc: 68.8375%
[0.01]
Train loss: 0.6792, Train acc: 74.9610%
Test loss: 0.5050  |  Test acc: 82.9832%
[0.01]
Train loss: 0.4922, Train acc: 83.0205%
Test loss: 0.3939  |  Test acc: 86.3445%
[0.01]
Train loss: 0.3493, Train acc: 88.4384%
Test loss: 0.3070  |  Test acc: 89.9160%
[0.01]
Train loss: 0.3026, Train acc: 89.9044%
Test loss: 0.4118  |  Test acc: 86.1345%


accuracy,▁▃▂▄▄▅▁▅▅▅▆▆▆▅▅▅▁▇▆▆▅▅▆▄▇▆▅█▇▇▆█▆▇▄▇▆▆▆█
average test batch loss,█▄▂▁▂
average test batch metric,▁▆▇█▇
average train batch loss,█▄▃▁▁
average train batch metric,▁▅▆██
epoch,▁▃▅▆█
loss,█▆▇▅▄▄▇▄▄▄▃▃▃▂▅▄█▂▃▃▅▂▂▅▃▂▃▁▂▂▂▁▄▂▃▂▃▃▂▁
test accuracy,▅▄▂▁▁▄▁▁▅▄▇▂▄▄▄▂▄▇▇▅▅▅▅▅▅▇▇▇▇▅▅▅▁▅▂▁██▇▅
test loss,▄▅▄█▅▄▆▆▆▄▃▅▅▄▆█▄▂▂▂▄▃▄▅▃▂▂▂▂▂▄▂▄▂▅▅▁▁▁▃
train time,▁
accuracy,0.8


...

wandb: Agent Starting Run: 3n41ytmq with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]
Train loss: 0.6146, Train acc: 78.5099%
Test loss: 0.2777  |  Test acc: 90.9896%
[0.01]
Train loss: 0.1372, Train acc: 95.7168%
Test loss: 0.1181  |  Test acc: 95.9896%
[0.01]
Train loss: 0.0809, Train acc: 97.4576%
Test loss: 0.1127  |  Test acc: 96.6840%
[0.001]
Train loss: 0.0308, Train acc: 99.2938%
Test loss: 0.0706  |  Test acc: 98.0208%
[0.001]
Train loss: 0.0188, Train acc: 99.6999%
Test loss: 0.0677  |  Test acc: 98.2292%


accuracy,▁▅▇▇▇▇▇▇▇▇███▇▇██▇████▇█▇███████████████
average test batch loss,█▃▂▁▁
average test batch metric,▁▆▇██
average train batch loss,█▂▂▁▁
average train batch metric,▁▇▇██
epoch,▁▃▅▆█
loss,█▅▃▂▂▂▂▂▂▂▁▁▁▂▂▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
test accuracy,▄█▇▁▂▂▅█▇█▇▂█▇█▇█▇█▇▅▇▇█▇██▅██▇█▅▅▇█▇█▇█
test loss,▅▃▃█▇▆▄▃▂▂▂▅▁▂▂▂▁▃▁▂▃▂▅▂▂▁▁▃▁▁▁▁▃▃▂▂▁▁▂▁
train time,▁
accuracy,1.0


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## inception_v3



In [ ]:
def train_model_inception_v3(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('inception_v3'))[-1]
        model = torch.hub.load('pytorch/vision', 'inception_v3', weight).to(device)
        model.fc = nn.Linear(2048 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# inception_v3 pipeline
sweep_id_inception_v3 = wandb.sweep(sweep_config, project="inception_v3-customData")
wandb.agent(sweep_id_inception_v3, train_model_inception_v3)

Create sweep with ID: b51zm6oo
Sweep URL: https://wandb.ai/gugi200/inception_v3-customData/sweeps/b51zm6oo


wandb: Agent Starting Run: bjvhnsu6 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: Ctrl + C detected. Stopping sweep.


## maxvit_t

In [ ]:
def train_model_maxvit_t(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('maxvit_t'))[-1]
        model = torch.hub.load('pytorch/vision', 'maxvit_t', weight).to(device)
        model.classifier[5] = nn.Linear(512 , len(class_names), bias=False).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# maxvit_t pipeline
sweep_id_maxvit_t = wandb.sweep(sweep_config, project="maxvit_t-customData")
wandb.agent(sweep_id_maxvit_t, train_model_maxvit_t)

Create sweep with ID: hqiytoj1
Sweep URL: https://wandb.ai/gugi200/maxvit_t-customData/sweeps/hqiytoj1


wandb: Agent Starting Run: xyd6i2r4 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://download.pytorch.org/models/maxvit_t-bc5ab103.pth" to /root/.cache/torch/hub/checkpoints/maxvit_t-bc5ab103.pth
100%|██████████| 119M/119M [00:01<00:00, 103MB/s] 


[0.01]
Train loss: 1.8840, Train acc: 26.0316%
Test loss: 1.6977  |  Test acc: 24.5082%
[0.01]
Train loss: 1.4697, Train acc: 35.5253%
Test loss: 1.4759  |  Test acc: 37.9010%
[0.01]
Train loss: 1.4409, Train acc: 37.7207%
Test loss: 1.4171  |  Test acc: 33.2400%
[0.01]
Train loss: 1.4106, Train acc: 38.5482%
Test loss: 1.3165  |  Test acc: 40.1861%
[0.01]
Train loss: 1.3717, Train acc: 39.6884%
Test loss: 1.3257  |  Test acc: 39.1873%


accuracy,▄▂▃▄▅▆▅▇▃▅▇▄▅▆▁▆▁▅▆█▅▅▅▇▅▅▇▇▅▂▃▆▄▅▇▄▅▆▃█
average test batch loss,█▄▃▁▁
average test batch metric,▁▇▅██
average train batch loss,█▂▂▂▁
average train batch metric,▁▆▇▇█
epoch,▁▃▅▆█
loss,█▅▅▅▄▃▃▃▃▂▄▃▂▄▆▃▄▃▂▁▅▃▃▂▅▃▁▃▃▃▃▂▃▃▂▃▃▃▄▁
test accuracy,▃▂▂▂▂▂▂▅▅▁▇▃▄▇▅█▆▂█▂▅▄▆▃▇▇▇▅▅▄▄▄▂▅▂█▂█▆▆
test loss,█▆▆▇█▇▆▄▂▆▄▅▅▃▅▃▄▄▂▆▁▅▃▅▃▃▃▃▅▂▅▃▄▂▅▂▆▁▃▃
train time,▁
accuracy,0.6


wandb: Agent Starting Run: tpm8vbh3 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]


## mobilenet_v3_large

In [8]:
def train_model_mobilenet_v3_large(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('mobilenet_v3_large'))[-1]
        model = torch.hub.load('pytorch/vision', 'mobilenet_v3_large', weight).to(device)
        model.classifier[3] = nn.Linear(1280 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# mobilenet_v3_large pipeline
sweep_id_mobilenet_v3_large = wandb.sweep(sweep_config, project="mobilenet_v3_large-customData")
wandb.agent(sweep_id_mobilenet_v3_large, train_model_mobilenet_v3_large)

Create sweep with ID: 2lxc33c8
Sweep URL: https://wandb.ai/gugi200/mobilenet_v3_large-customData/sweeps/2lxc33c8


wandb: Agent Starting Run: e3wt8r8b with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Using cache found in /root/.cache/torch/hub/pytorch_vision_main
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:02<00:00, 7.51MB/s]


[0.01]
Train loss: 1.3500, Train acc: 53.6225%
Test loss: 76.3980  |  Test acc: 16.0487%
[0.01]
Train loss: 0.7980, Train acc: 72.2675%
Test loss: 12.1116  |  Test acc: 21.6707%
[0.01]
Train loss: 0.6078, Train acc: 79.2127%
Test loss: 1.3447  |  Test acc: 68.9846%
[0.01]
Train loss: 0.4778, Train acc: 83.9554%
Test loss: 1.1019  |  Test acc: 73.0251%
[0.01]
Train loss: 0.4494, Train acc: 85.0519%
Test loss: 15.2876  |  Test acc: 30.8944%


accuracy,▁▃▁▂▅▅▅▄▅▆▂▂▅▆▅▇▃▆▅▅▅▅▆▄▆▅▇▇██▆▇▅▅██▅▆██
average test batch loss,█▂▁▁▂
average test batch metric,▁▂██▃
average train batch loss,█▄▂▁▁
average train batch metric,▁▅▇██
epoch,▁▃▅▆█
loss,█▆▆▆▃▃▃▇▅▃▆▆▃▃▃▂▄▃▄▄▄▄▂▄▂▃▂▂▂▂▃▂▅▄▁▁▄▂▁▂
test accuracy,▃▃▄▂▃▃▃▃▂▂▁▄▂▂▁▃▇▇▆▇▆▆▇▇▇▆▇▇█▇▇▆▃▄▄▄▃▃▃▄
test loss,█▆▆▆▅▆▆▇▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂
train time,▁
accuracy,0.73333


...

wandb: Agent Starting Run: a44jlxyp with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]
Train loss: 0.6441, Train acc: 76.8715%
Test loss: 1.2034  |  Test acc: 56.5855%
[0.01]
Train loss: 0.1554, Train acc: 95.0388%
Test loss: 0.6208  |  Test acc: 78.0014%
[0.01]
Train loss: 0.0703, Train acc: 97.6671%
Test loss: 0.1887  |  Test acc: 94.0572%
[0.001]
Train loss: 0.0361, Train acc: 98.8854%
Test loss: 0.0778  |  Test acc: 97.8743%
[0.001]
Train loss: 0.0191, Train acc: 99.4856%
Test loss: 0.0735  |  Test acc: 97.5565%


accuracy,▁▂▅▇▆██▇▇▇▇████▇███████████████▇████████
average test batch loss,█▄▂▁▁
average test batch metric,▁▅▇██
average train batch loss,█▃▂▁▁
average train batch metric,▁▇▇██
epoch,▁▃▅▆█
loss,█▆▄▂▃▂▂▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
test accuracy,▃▄▂▃▃▃▁▃▆▄▆▇▅▅▅▅▇▇▇▇▇█▇▇█████▇█▇███▇▇███
test loss,▆▅█▇▇▆█▆▄▄▃▃▃▄▄▃▂▂▂▂▁▁▂▃▁▁▁▂▁▂▁▂▁▁▁▂▂▁▁▁
train time,▁
accuracy,0.93333


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## shufflenet_v2_x2_0

In [7]:
def train_model_shufflenet_v2_x2_0(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('shufflenet_v2_x2_0'))[-1]
        model = torch.hub.load('pytorch/vision', 'shufflenet_v2_x2_0', weight).to(device)
        model.fc = nn.Linear(2048 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# shufflenet_v2_x2_0 pipeline
sweep_id_shufflenet_v2_x2_0 = wandb.sweep(sweep_config, project="shufflenet_v2_x2_0-customData")
wandb.agent(sweep_id_shufflenet_v2_x2_0, train_model_shufflenet_v2_x2_0)

Create sweep with ID: 05tdx6c8
Sweep URL: https://wandb.ai/gugi200/shufflenet_v2_x2_0-customData/sweeps/05tdx6c8


wandb: Agent Starting Run: ey7xjnqn with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Using cache found in /root/.cache/torch/hub/pytorch_vision_main
Downloading: "https://download.pytorch.org/models/shufflenetv2_x2_0-8be3c8ee.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x2_0-8be3c8ee.pth
100%|██████████| 28.4M/28.4M [00:00<00:00, 93.4MB/s]


[0.01]
Train loss: 0.9614, Train acc: 63.7748%
Test loss: 2.7928  |  Test acc: 37.3033%
[0.01]
Train loss: 0.4867, Train acc: 82.9828%
Test loss: 0.4324  |  Test acc: 85.0863%
[0.01]
Train loss: 0.3429, Train acc: 88.1905%
Test loss: 0.2992  |  Test acc: 90.2542%
[0.01]
Train loss: 0.2471, Train acc: 91.7847%
Test loss: 0.3218  |  Test acc: 88.9149%
[0.01]
Train loss: 0.2338, Train acc: 92.0326%
Test loss: 0.4983  |  Test acc: 83.3913%


accuracy,▁▄▃▃▅▅▇▇▇▆▇▆▇▇▆▇▆▆▇█▇▆▇▆▆▇▇▇█▇▇█▇▇▆▇▇▇▇▆
average test batch loss,█▁▁▁▂
average test batch metric,▁▇██▇
average train batch loss,█▃▂▁▁
average train batch metric,▁▆▇██
epoch,▁▃▅▆█
loss,█▆▅▅▄▄▄▂▂▃▂▃▂▂▃▂▂▃▁▁▂▃▂▂▂▂▁▂▁▁▁▁▁▂▃▂▁▁▂▂
test accuracy,▃▃▄▄▃▂▃▁▆▇▇▆▇▇█▇▆█▇▆▇▇▇▇▇▇▇▇█▇▇█▆▇▆▇▆▆▇▆
test loss,▆▆▅▄▅▅█▇▂▂▁▂▃▁▁▁▂▁▁▂▁▄▁▂▁▁▁▁▁▁▁▁▁▂▂▁▂▃▁▁
train time,▁
accuracy,1.0


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: b014oenh with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]
Train loss: 0.6890, Train acc: 76.1782%
Test loss: 0.7649  |  Test acc: 86.9633%
[0.01]
Train loss: 0.1369, Train acc: 95.9922%
Test loss: 0.5793  |  Test acc: 85.5191%
[0.01]
Train loss: 0.0542, Train acc: 98.3581%
Test loss: 3.3261  |  Test acc: 95.2260%
[0.001]
Train loss: 0.0210, Train acc: 99.4174%
Test loss: 0.9799  |  Test acc: 97.0304%
[0.001]
Train loss: 0.0138, Train acc: 99.8058%
Test loss: 2.1766  |  Test acc: 96.9209%


accuracy,▁▂▆▆▇▆████▇█▇▇▇▇████████████████████████
average test batch loss,▁▁█▂▅
average test batch metric,▂▁▇██
average train batch loss,█▂▁▁▁
average train batch metric,▁▇███
epoch,▁▃▅▆█
loss,█▇▄▄▂▃▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test accuracy,▅▂▅▇▃▂▁▃▆▅▁▅▆▄▄▄▇▇█▇▄▆█▅████▇▆██▇▇▆▇█▆▇▇
test loss,▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▂▁▁▁█▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁
train time,▁
accuracy,1.0


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## swin_v2_t

In [7]:
def train_model_swin_v2_t(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('swin_v2_t'))[-1]
        model = torch.hub.load('pytorch/vision', 'swin_v2_t', weight).to(device)
        model.head = nn.Linear(768 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# swin_v2_t pipeline
sweep_id_swin_v2_t = wandb.sweep(sweep_config, project="swin_v2_t-customData")
wandb.agent(sweep_id_swin_v2_t, train_model_swin_v2_t)

Create sweep with ID: iuip7fci
Sweep URL: https://wandb.ai/gugi200/swin_v2_t-customData/sweeps/iuip7fci


wandb: Agent Starting Run: z08d9ljc with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Using cache found in /root/.cache/torch/hub/pytorch_vision_main
Downloading: "https://download.pytorch.org/models/swin_v2_t-b137f0e2.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_t-b137f0e2.pth
 11%|█         | 11.6M/109M [00:00<00:01, 66.3MB/s]wandb: Ctrl + C detected. Stopping sweep.
 22%|██▏       | 23.8M/109M [00:00<00:01, 54.3MB/s]

## vgg19_bn

In [6]:
def train_model_vgg19_bn(config=None):
    with wandb.init(config=config):


        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        config = wandb.config

        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('vgg19_bn'))[-1]
        model = torch.hub.load('pytorch/vision', 'vgg19_bn', weight).to(device)
        model.classifier[6] = nn.Linear(4096 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# vgg19_bnvgg19_bn pipeline
sweep_id_vgg19_bn = wandb.sweep(sweep_config, project="vgg19_bn-customData")
wandb.agent(sweep_id_vgg19_bn, train_model_vgg19_bn)

Create sweep with ID: 64dxa04w
Sweep URL: https://wandb.ai/gugi200/vgg19_bn-customData/sweeps/64dxa04w


wandb: Agent Starting Run: ljq0vl37 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: Currently logged in as: ad20781 (gugi200). Use `wandb login --relogin` to force relogin


Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:10<00:00, 56.6MB/s]


[0.01]
Train loss: 4.4189, Train acc: 17.0904%
Test loss: 2.0095  |  Test acc: 18.5154%
[0.01]
Train loss: 2.7558, Train acc: 17.5661%
Test loss: 2.0220  |  Test acc: 17.5015%
[0.01]
Train loss: 2.0114, Train acc: 18.7347%
Test loss: 1.9553  |  Test acc: 22.9570%
[0.01]
Train loss: 1.9730, Train acc: 19.3532%
Test loss: 1.9238  |  Test acc: 23.3883%
[0.01]
Train loss: 1.9502, Train acc: 20.2609%
Test loss: 1.8040  |  Test acc: 24.9697%


accuracy,▃▇▁▆▃▅▂▇▂▂▆▃▅▆▂▅▇█▅▇▇▂▆▅▃▇▃▃▇▇▃▅▆▆▆▃▃▆▅▇
average test batch loss,██▆▅▁
average test batch metric,▂▁▆▇█
average train batch loss,█▃▁▁▁
average train batch metric,▁▂▅▆█
epoch,▁▃▅▆█
loss,▃▃▃▂▂▂▂▁▂▁▂▂▁▁█▂▁▁▂▂▁▂▁▁▂▂▂▂▁▂▂▂▁▂▂▂▃▂▂▁
test accuracy,▁▅▂▃▆▃▄▄▃▃▄▅▃▄▄▃▅▅▇▄▁▅▄▄█▆▇▄█▅▃▃▃▃▅▆▇█▇▆
test loss,▇▆▇▄▆▇▆▄▆▇█▄▇▅▄▆▇▄▃▅▅▆▅█▃▅▅▄▃▄▇▆▄▄▃▁▄▁▄▂
train time,▁
accuracy,0.33333


...

wandb: Agent Starting Run: z0uafm3i with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]
Train loss: 0.6219, Train acc: 78.8089%
Test loss: 0.2084  |  Test acc: 93.1568%
[0.01]
Train loss: 0.2436, Train acc: 92.6201%
Test loss: 0.1253  |  Test acc: 95.9075%
[0.01]
Train loss: 0.1039, Train acc: 96.7691%
Test loss: 0.1858  |  Test acc: 94.3291%
[0.01]
Train loss: 0.0658, Train acc: 98.0226%
Test loss: 0.0752  |  Test acc: 97.8284%
[0.001]
Train loss: 0.0227, Train acc: 99.2938%
Test loss: 0.0499  |  Test acc: 98.7253%


accuracy,▁▅▅▆▆▆▆▇█▇▇▇▇████▇██████▇█▇███▇█████████
average test batch loss,█▄▇▂▁
average test batch metric,▁▄▂▇█
average train batch loss,█▄▂▂▁
average train batch metric,▁▆▇██
epoch,▁▃▅▆█
loss,█▇▅▃▃▄▃▂▂▂▃▂▂▁▁▁▁▂▁▁▁▁▁▁▃▁▁▁▁▂▂▁▁▁▁▁▁▂▁▁
test accuracy,▅▄▅▇▇▁▇▄█▇████▆▅▆▆█▆▆█▅▆█▆▆█▇▆▇██▆██▇██▇
test loss,▄▆▃▃▂█▃▄▁▁▁▂▁▁▃▃▅▃▁▅▂▁▅▂▁▂▃▁▂▄▂▁▁▆▁▁▂▁▁▂
train time,▁
accuracy,1.0


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## wide_resnet50_2

In [6]:
def train_model_wide_resnet50_2(config=None):
    with wandb.init(config=config):
        config = wandb.config

        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]
        tranform = transforms.Compose([
            expMapping(4),
            softThreshold(30, 255),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])


        train_dataloader, class_names, _ = create_dataset(path=Path("train_raw"),
                                                          batchsize=config.batch_size,
                                                          preprocess=tranform)

        test_dataloader, _ , _ = create_dataset(path=Path("test_raw"),
                                                batchsize=config.batch_size,
                                                preprocess=tranform)


        weight = list(torchvision.models.get_model_weights('wide_resnet50_2'))[-1]
        model = torch.hub.load('pytorch/vision', 'wide_resnet50_2', weight).to(device)
        model.fc = nn.Linear(2048 , len(class_names), bias=True).to(device)

        train_test_loop(config=config,
                        model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        class_names=class_names,
                        device=device
                        )
    return model


# wide_resnet50_2 pipeline
sweep_id_wide_resnet50_2 = wandb.sweep(sweep_config, project="wide_resnet50_2-customData")
wandb.agent(sweep_id_wide_resnet50_2, train_model_wide_resnet50_2)

Create sweep with ID: pogqeb71
Sweep URL: https://wandb.ai/gugi200/wide_resnet50_2-customData/sweeps/pogqeb71


wandb: Agent Starting Run: iljgg8ud with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: Currently logged in as: ad20781 (gugi200). Use `wandb login --relogin` to force relogin


Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-9ba9bcbe.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-9ba9bcbe.pth
100%|██████████| 263M/263M [00:21<00:00, 12.6MB/s]


[0.01]
Train loss: 1.2501, Train acc: 52.9391%
Test loss: 0.7589  |  Test acc: 68.2773%
[0.01]
Train loss: 0.6143, Train acc: 78.2519%
Test loss: 0.6066  |  Test acc: 77.3109%
[0.01]
Train loss: 0.4430, Train acc: 84.3957%
Test loss: 0.3532  |  Test acc: 88.4454%
[0.01]
Train loss: 0.3176, Train acc: 89.3177%
Test loss: 0.3254  |  Test acc: 89.5483%
[0.01]
Train loss: 0.2567, Train acc: 91.3598%
Test loss: 0.4014  |  Test acc: 87.3424%


accuracy,▁▄▂▂▅▅▅▆▄▅▆▆▇▆▅▇▇▇▇▆▇▇▇█▆▇▆▇▆▆▇▇▇█▇▇▇▆██
average test batch loss,█▆▁▁▂
average test batch metric,▁▄██▇
average train batch loss,█▄▂▁▁
average train batch metric,▁▆▇██
epoch,▁▃▅▆█
loss,█▇▇▇▅▄▅▃▅▃▃▄▂▃▃▃▃▂▂▂▂▂▂▁▃▁▃▂▃▃▃▂▂▁▂▂▂▂▁▁
test accuracy,▂▂▂▅▂▅▃▆▅▁▆▆▅▆▆▄█▇▅▇▆▅▄▅█▅▄█▅▅▇▇▄▇▇▅▆█▅▆
test loss,▄▄▅▅▅▄▃▃▂▅▂▂▃▂▂▃▁▂▂▂█▄▄▂▁▄▃▁▂▃▂▂▄▃▂▂▂▁▅▃
train time,▁
accuracy,1.0


...

accuracy,▁▂▃▃▃▃▄▅▃▄▅▇▅▆▅▆▄▆▆▆▅▅▇▅▆▇▇▆█▇█▆█▇▆▆██▇▇
average test batch loss,▁█▁▁▁
average test batch metric,▅▁▇▇█
average train batch loss,█▄▃▁▁
average train batch metric,▁▅▆██
epoch,▁▃▅▆█
loss,█▆▆▆▅▅▄▅▄▄▃▂▄▃▄▃▆▂▂▃▃▃▂▅▃▂▂▃▁▂▁▂▁▂▃▃▁▁▁▂
test accuracy,▄▅▃▅▄▅▆▄▂▂▂▁▁▂▁▂▆▇▆▅▆▇▆▆▆▇▆▆▆▆▆▆█▇▆▇▆▇▇█
test loss,▁▁▁▁▁▁▁▁▇▇▇█▇▆█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▁
accuracy,0.66667


wandb: Agent Starting Run: xnvyj0g0 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	fc_layer_size: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


[0.01]
Train loss: 0.6627, Train acc: 76.6066%
Test loss: 0.2387  |  Test acc: 92.6042%
[0.01]
Train loss: 0.1131, Train acc: 96.3100%
Test loss: 0.1159  |  Test acc: 96.0417%
[0.01]
Train loss: 0.0417, Train acc: 98.7818%


wandb: Ctrl + C detected. Stopping sweep.
